## 설치

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 4.6 MB/s 
     |████████████████████████████████| 163 kB 87.9 MB/s 
     |████████████████████████████████| 7.6 MB 81.4 MB/s 


In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 4.6 MB/s 
     |████████████████████████████████| 212 kB 72.1 MB/s 
     |████████████████████████████████| 115 kB 84.2 MB/s 
     |████████████████████████████████| 127 kB 85.2 MB/s 
     |████████████████████████████████| 115 kB 90.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
!pip install wandb -qU

     |████████████████████████████████| 1.9 MB 4.7 MB/s 
     |████████████████████████████████| 166 kB 86.2 MB/s 
     |████████████████████████████████| 182 kB 81.9 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 166 kB 90.8 MB/s 
     |████████████████████████████████| 162 kB 95.7 MB/s 
     |████████████████████████████████| 162 kB 97.5 MB/s 
     |████████████████████████████████| 158 kB 93.9 MB/s 
     |████████████████████████████████| 157 kB 92.9 MB/s 
     |████████████████████████████████| 157 kB 94.4 MB/s 
     |████████████████████████████████| 157 kB 97.1 MB/s 
     |████████████████████████████████| 157 kB 82.4 MB/s 
     |████████████████████████████████| 157 kB 97.3 MB/s 
     |████████████████████████████████| 157 kB 81.6 MB/s 
     |████████████████████████████████| 157 kB 95.1 MB/s 
     |████████████████████████████████| 156 kB 96.2 MB/s 


In [4]:
!kaggle datasets download -d minjaechoi99/aihub-groom

 96% 438M/457M [00:02<00:00, 244MB/s]
100% 457M/457M [00:02<00:00, 202MB/s]


In [5]:
!unzip /content/aihub-groom.zip

Archive:  /content/aihub-groom.zip
  inflating: data/Training/talksets-train-1.json  
  inflating: data/Training/talksets-train-2.json  
  inflating: data/Training/talksets-train-3.json  
  inflating: data/Training/talksets-train-4.json  
  inflating: data/Training/talksets-train-5.json  
  inflating: data/Validation/talksets-train-6.json  
  inflating: modelSource/class.json  
  inflating: modelSource/data/test   
  inflating: modelSource/data/test5  
  inflating: modelSource/data/train  
  inflating: modelSource/data/train5  
  inflating: modelSource/data/val    
  inflating: modelSource/data/val5   
  inflating: modelSource/emodel-class.log  
  inflating: modelSource/model/README.md  
  inflating: modelSource/model/config.json  
  inflating: modelSource/model/pytorch_model.bin  
  inflating: modelSource/model/tokenizer_config.json  
  inflating: modelSource/model/vocab.txt  
  inflating: modelSource/multi.json  
  inflating: modelSource/processor.py  
  inflating: modelSource/requir

In [6]:
import torch
import torch.optim as optim

import numpy as np
import tqdm as tqdm

from transformers import(
    AutoModel,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    default_data_collator,
    EarlyStoppingCallback,
    get_cosine_schedule_with_warmup,
    get_constant_schedule,
    AdamW
)

In [7]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## readjson

In [55]:
from pathlib import Path
import json, re
def read_aihub(path):
    path = Path(path)
    with open(path, 'rb') as f:
        print(f)
        squad_dict = json.load(f)
    texts = []
    intensities = []
    for group in squad_dict:
        for passage in group['sentences']:
            text = passage['origin_text']
            text = re.sub('#.*?#', '[UNK]', text)
            intensity = passage['intensity']
            if intensity == 1:
                continue
            texts.append(text)
            if intensity < 1:
                intensity = 0
            else:
                intensity = 1
            intensities.append(intensity)

    return texts ,intensities

In [56]:
import pandas as pd

file_path =  '/content/data/Training/talksets-train-1.json'

texts , intensities = read_aihub(file_path)
ai_df = pd.DataFrame(list(zip(texts , intensities)),
               columns =['texts','intensities'])

#file_path = '/content/data/Validation/talksets-train-6.json'

#texts , intensities = read_aihub(file_path)
#test_df = pd.DataFrame(list(zip(texts , intensities)),
#               columns =['texts','intensities'])

<_io.BufferedReader name='/content/data/Training/talksets-train-1.json'>


In [57]:
ai_df

,texts,intensities
0,부랴부랴 왔는데 아무도 안왔네. 시간개념들이 없네,1
1,맞아. 사람들이 진짜 개념이없다,1
2,저렇게는 살지 말아야지,1
3,그러게 게으르고 멍청한 사람들은 맞아야해,1
4,특히 벗방보는 애들은 진짜 거세 시켜야함,1
...,...,...
54172,앰생이 같은 게 같이 다녀줬더니 내가 니 친구로 보이냐?,1
54173,누가 누굴 데리고 다녀 나사 빠진 놈이,1
54174,너 오늘 나한테 처갈피는 줄 알아라,1
54175,엄친아들 다 없애버리고 싶다,1


In [58]:
ai_df = ai_df.sample(frac=1,random_state=43)

In [59]:
spliter = int(len(ai_df)*0.8)

In [60]:
train_df = ai_df.iloc[:spliter,:]
dev_df = ai_df.iloc[spliter:,:]

In [61]:
dev_df

,texts,intensities
53436,뭔 신박한 헛소리를하지..코로나에 안전한곳이 없다며??,1
7444,자기가 아직도 총리할 때 지지율인 줄 아나 봄,0
3630,승무원들은 다 몸 파는 애들인듯,1
37547,난 페미니스트 지지하지만 저런 여자들은 보면 진짜 웃기더라,0
48421,중국놈들은 하나같이 눈이 작아서 볼품없어보여.,1
...,...,...
18448,왜 나한테만 이런 식으로 구는 거야?,0
40753,저러니까 지나가는 사람들한테도 손가락질을 당하지.,1
51455,"응, 오래 서 있고 그럴땐 다리 아프니까 플랫으로 바꾸려고",0
19776,나한테는 어떤 옷이 잘 어울려?,0


In [62]:
train_df

,texts,intensities
21368,니가 만난 남자들이 다 더러운 거겠지,1
38161,니네 누나 육변기 된 거 같던데?,1
32313,쟤는 현실자각도 못 하는 저런 멍청한 머리로 대체 어떻게 사냐,1
48441,어휴 [UNK] 저 개눈알 영감은 죽지도 않고 다 해 처먹어,1
7593,자기 여친보고 낙태 종용했대,0
...,...,...
50755,응 내가 하는데 왜?,0
36378,짐승만도 못한것들 진짜 무슨 생각으로 저렇게 쓰레기짓을 했을까,0
54132,내 롤모델 [UNK]야,1
10234,야 옆 차선에 차도 없는데 좀 이동하자,0


In [63]:
from datasets import Dataset

train_data = Dataset.from_pandas(train_df)
dev_data = Dataset.from_pandas(dev_df)
dev_data

Dataset({
    features: ['texts', 'intensities', '__index_level_0__'],
    num_rows: 10836
})

In [64]:
train_data

Dataset({
    features: ['texts', 'intensities', '__index_level_0__'],
    num_rows: 43341
})

## tokenize

In [65]:
model_name = 'beomi/KcELECTRA-base-v2022'
max_length = 128

In [66]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--beomi--KcELECTRA-base-v2022/snapshots/4431b6c7ad00f82fd50880864574cef97e0a368b/config.json
Model config ElectraConfig {
  "_name_or_path": "beomi/KcELECTRA-base-v2022",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "tokenizer_class": "BertTokenizer",
  "transformers_version": "4.23.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 

In [67]:
def tokenizeWithLabel(data):
    tokenized_datas = tokenizer(
        data['texts'],
        max_length=max_length,
        padding="max_length",
        truncation="only_second"
    )
    tokenized_datas['labels'] = data['intensities']
    return tokenized_datas

In [68]:
train_tokenized_datasets = train_data.map(tokenizeWithLabel, batched=True, remove_columns=train_data.column_names)
dev_tokenized_datasets = dev_data.map(tokenizeWithLabel, batched=True, remove_columns=dev_data.column_names)

  0%|          | 0/44 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

In [69]:
train_tokenized_datasets

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 43341
})

## train with sweep

In [78]:
sweep_configuration = {
    'method': 'grid',
    'name': 'sweep',
    'metric': {'goal': 'minimize', 'name': 'eval/loss'},
    'parameters': 
    {
        'batch_size': {'values': [64]},
        'epochs': {'values': [10]},
        'lr': {'values': [5e-5]}
     }
}

In [79]:
max_batch_size = 64
def train():
    torch.cuda.empty_cache()
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    grouped_params = model.parameters()
    run = wandb.init(config=sweep_configuration, entity="groom2team")
    batch_size = max_batch_size
    gradient_accumulation_steps= wandb.config.batch_size // max_batch_size
    epochs = wandb.config.epochs
    total_steps = int(len(train_tokenized_datasets)/wandb.config.batch_size*epochs)
    learning_rate = wandb.config.lr
    data_collator = default_data_collator
    grouped_params = model.parameters()
    optimizer=AdamW(grouped_params, lr=learning_rate)
    scheduler=get_cosine_schedule_with_warmup(optimizer=optimizer,
                                              num_warmup_steps=total_steps//(epochs*3),
                                              num_training_steps=total_steps+1)
    #scheduler=get_constant_schedule(optimizer=optimizer)
    optimizers = optimizer, scheduler
    args = TrainingArguments(
        f"{model_name}-finetuned",
        evaluation_strategy = "steps",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=epochs,
        gradient_accumulation_steps = gradient_accumulation_steps,
        report_to="wandb",
        run_name="utopia",
        logging_steps = total_steps//200,
        eval_steps = total_steps//100,
        save_steps = total_steps//100,
        weight_decay=0.0,
        save_total_limit = 2,
        load_best_model_at_end=True
    )
    trainer = Trainer(
        model,
        args,
        train_dataset=train_tokenized_datasets,
        eval_dataset=dev_tokenized_datasets,
        data_collator=data_collator,
        tokenizer=tokenizer,
        callbacks = [EarlyStoppingCallback(early_stopping_patience=10)],
        optimizers=optimizers
    )
    trainer.train()# train 하고
    trainer.save_model(output_dir= 'pytorch_finetuned') # trainer에서 실행된 model save
    artifact = wandb.Artifact(name='pytorch_finetuned', type='model') # wandb에 해당 모델 version 관리.
    artifact.add_dir('pytorch_finetuned', name='best_model_at_end')
    run.log_artifact(artifact)

In [80]:
sweep_id = wandb.sweep(sweep=sweep_configuration, project='aihub_pj3', entity='groom2team')
count = 1

Create sweep with ID: gw303mal
Sweep URL: https://wandb.ai/groom2team/aihub_pj3/sweeps/gw303mal


In [ ]:
wandb.agent(sweep_id, function=train, count=count)

wandb: Agent Starting Run: t3yl3e1h with config:
wandb: 	batch_size: 64
wandb: 	epochs: 10
wandb: 	lr: 5e-05
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--beomi--KcELECTRA-base-v2022/snapshots/4431b6c7ad00f82fd50880864574cef97e0a368b/config.json
Model config ElectraConfig {
  "_name_or_path": "beomi/KcELECTRA-base-v2022",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "tokenizer_class": "Bert

PyTorch: setting up devices
***** Running training *****
  Num examples = 43341
  Num Epochs = 10
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 6780
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss,Validation Loss
67,0.629700,0.519958


***** Running Evaluation *****
  Num examples = 10836
  Batch size = 64
Saving model checkpoint to beomi/KcELECTRA-base-v2022-finetuned/checkpoint-67
Configuration saved in beomi/KcELECTRA-base-v2022-finetuned/checkpoint-67/config.json
Model weights saved in beomi/KcELECTRA-base-v2022-finetuned/checkpoint-67/pytorch_model.bin
tokenizer config file saved in beomi/KcELECTRA-base-v2022-finetuned/checkpoint-67/tokenizer_config.json
Special tokens file saved in beomi/KcELECTRA-base-v2022-finetuned/checkpoint-67/special_tokens_map.json
Deleting older checkpoint [beomi/KcELECTRA-base-v2022-finetuned/checkpoint-1080] due to args.save_total_limit


In [74]:
wandb.finish() # wandb 종료

AttributeError: ignored